In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#
# A library that provides a Foslipy interface to the Telegram Bot API
# Copyright (C) 2015-2018
# Shivam Gupta <pyshivam.py@gmail.com>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser Public License for more details.
#
# You should have received a copy of the GNU Lesser Public License
# along with this program.  If not, see [http://www.gnu.org/licenses/].

from threading import Thread
import time

from telegram import Bot, Update, ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, MessageHandler, CommandHandler, CallbackQueryHandler
from telegram.ext.filters import Filters
from pymongo import MongoClient

custom_keyboard = [['/Workshop - to get details about Workshop.'],
                   ['/ITP - to get details about ITP.'],
                   ['/Internship - to get details about Internship.'],
                   ['/Project_Consultation - to get details about Project Consultation.'],
                   ['/Interview - to get details about Interview Prepare.'],
                   ['/Placement - to get details about Placement Assist.'],
                   ['/Chat_bot - to get details about Chat Bot.']]
reply_markup = ReplyKeyboardMarkup(custom_keyboard)
reply_markup1 = ReplyKeyboardRemove()


#########################
# All Classes Goes Here #
#########################

# Creating class for all operation related to bot.
class DataBase:
    def __init__(self, database='users', table='users_id'):
        self.client = MongoClient()
        self.db = self.client[database]  # 'users'
        self.table = self.db[table]  # 'users_id'

    def check_user(self, chat_id=None, username=None):
        if chat_id:
            try:
                details = self.table.find_one({'_id': chat_id})
                if details:
                    return True, details
                else:
                    return False, "User is not present in the database."
            except Exception as e:
                return False, e
        elif username:
            user = self.table.find_one({'username': username})
            if user:
                return True, user['_id']
            else:
                return False, "User not Present"

    def add_user(self, chat_id, name, username):
        try:
            self.table.insert_one({'_id': chat_id, 'name': name, 'username': username})
            return True,
        except Exception as e:
            return False, e

    def update_user(self, chat_id, *parameters):
        pass

    def remove_user(self, chat_id):
        try:
            self.table.remove({'_id': chat_id})
            return True,
        except Exception as e:
            return False, e

    def find(self, broadcast=False):
        if not broadcast:
            users = "\n\n".join(
                [str(
                    "Chat id = " + str(a['_id']) + "\nUsername = " + str(a['username'] + "\nName = " + str(a['name']))
                ) for a in self.table.find()])
            # print(users)
            return users
        else:
            chat_ids = list()
            for user in self.table.find():
                chat_id = user['_id']
                chat_ids.append(chat_id)
            return chat_ids


##############
# Admin Area #
##############

# /start threaded admin handler
def threaded_admin_start(bot, update):
    custom_keyboard1 = [['/start - To start the bot.'], ['/users - To get details about Users.']]
    rm = ReplyKeyboardMarkup(custom_keyboard1)
    chat_id = update.message.chat_id
    bot.send_message(chat_id=chat_id, text="""
Control Panel 
/start - Normal start command to bot.
/users - to get details about users.
/msg - Send Message to particular user.
        e.g /msg <username> <Message>
/broadcast - Send Message Message to all.
        e.g /broadcast <Message>
""", reply_markup=rm)


# /start received by admin of this bot.
def admin_start(bot: Bot, update: Update):
    start_admin = Thread(target=threaded_admin_start, args=(bot, update))
    start_admin.start()


# Send List of Users to Admin.
def send_users(bot: Bot):
    bot.send_message(chat_id=493497879, text=db.find())


# Send Message to Particular User.
def send_msg(bot: Bot, update: Update):
    username = update.message.text.split()[1]
    msg = ' '.join(update.message.text.split()[2:])
    try:
        chat_id = int(username)
        result = db.check_user(chat_id=chat_id)
    except Exception as e:
        print(e)
        result = db.check_user(username=username)
    if result[0]:
        bot.send_message(chat_id=493497879, text="Message sent to: %s" % username)
        chat_id = result[1]["_id"]
        bot.send_message(chat_id=chat_id, text=msg)
    else:
        bot.send_message(chat_id=493497879, text=result[1])


# Broadcast message to all users.
def broadcast_msg(bot: Bot, update: Update):
    msg = " ".join(update.message.text.split()[1:])
    chat_ids = db.find(broadcast=True)
    for chat_id in chat_ids:
        bot.send_message(chat_id=chat_id, text=msg)


# When admin send command then this function will handle commands.
def admin(bot: Bot, update: Update):
    text = update.message.text
    if text.startswith("/users"):
        Thread(target=send_users, args=(bot,)).start()
    elif text.startswith("/msg"):
        Thread(target=send_msg, args=(bot, update)).start()
    elif text.startswith("/broadcast"):
        Thread(target=broadcast_msg, args=(bot, update)).start()

    # print("welcome Admin")


############
# services #
############

# callback handler for fly button.
def button(bot: Bot, update: Update) -> None:
    query = update.callback_query
    if query.data == 'workshop student':
        bot.send_message(chat_id=query.message.chat_id, text="""
Students can attend 1 or 2 days workshop organised by the college in their premises.
Student can also attend the workshop organised at Foslipy Future Lab office.
Workshop are :-
Click on workshop to download content

[Foss](http://www.foslipy.com/workshops/free-and-open-source-workshop/)

[Python](http://www.foslipy.com/workshops/python-basics/)

[Python Adv Framework Flask n Django](http://www.foslipy.com/workshops/advaned-python-framework/)

[PHP Adv Framework Laravel](http://www.foslipy.com/workshops/php-mysql/)

[PHP with CMS](http://www.foslipy.com/workshops/content-management-system/)

[IOT Raspberry Pi](www.foslipy.com/)

[ERP : Enterprise Resource Planning](http://www.foslipy.com/workshops/web-application-frameworkerp/)

[LATEX](http://www.foslipy.com/workshops/latex-a-document-preparation-system/ )

[Android](http://www.foslipy.com/workshops/android-application/)

[Cloud Computing](http://www.foslipy.com/workshops/cloud-computing/)

[BigData](http://www.foslipy.com)

""", parse_mode='Markdown')
    elif query.data == 'workshop teacher':
        bot.send_message(chat_id=query.message.chat_id, text="""
FDP is organised for teachers in colleges.
Workshop are :-
Click on workshop to download content

Python for Data Analytics

Big data with Hadoop

R Programming

""")
    elif query.data == 'ITP python':
        bot.send_message(chat_id=query.message.chat_id, text="""
Python Adv Frameworks - Students will learn Advance frameworks like Flask and Django. Overview of Frappe framework.

They will do a project in the last two months using the framework.

Students can use the work done as their final year Project.

Students will learn -
Python fundamentals

Data Structure Algorithms - list, tuples & dictionaries

Object orientation concepts

GUI programming - Tkinter

Adv MVC Web framework Flask and Django

Web Application Development

Data Analytics libraries

Industry based projects on Python

""")
    elif query.data == 'ITP php-python':
        bot.send_message(chat_id=query.message.chat_id, text="""
Python & PHP Integration:- 
Students will learn two python Framework Flask & Django and then Php framework Laravel with WordPress and Drupal.

They will do the project in the last two months using both the technologies(Python and PHP).

Students can use the work done as their final year Project.

Students will learn -
Python and PHP fundamentals

Data Structure Algorithms - List, Tuples & Dictionaries

Object orientation concepts

Data Analytics libraries using Python and PHP

Adv MVC Web framework Flask and Django

Adv MVC Web framework Laravel

Web Application Development

Industry based projects on Python

""")
    elif query.data == 'internship 6-month':
        bot.send_message(chat_id=query.message.chat_id, text="""
It is given to BSc, MSc, BCA, MCA Students.

Students will be given training on Python and PHP technologies for 2 months.

After successful completion of training Students will get the opportunity to work on Live project.

After Submit the projects and on successful completion,

participant will receive 6 months experience certificate.

""")
    elif query.data == 'internship 1-year':
        bot.send_message(chat_id=query.message.chat_id, text="""
It is given to Engineering students of BE.

Training & Mentor will be given to students and then they can work on Company Live projects.

Students will learn two frameworks of Python that is Flask and Django and PHP framework 
Laravel with overview of WordPress and Drupal.

Students will have to Develop a Complex Web Application and an Android App supporting the web Application.

After Submit the projects and on successful completion participant will receive 1Year experience certificate. 

""")


# chat bot
def chat_bot(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    bot.send_message(chat_id=chat_id, text="Now your taking to chatBot.", reply_markup=reply_markup1)


# Give details about workshop.
def workshop(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    keyboard = [[InlineKeyboardButton("Student", callback_data='workshop student'),
                 InlineKeyboardButton("Teacher", callback_data='workshop teacher')]]
    wk = InlineKeyboardMarkup(keyboard)
    bot.send_message(chat_id=chat_id,
                     text="Organised in Colleges and at our Foslipy Office.\nWorkshop details for: ", reply_markup=wk)


# Give details about Industrial Training Program.
def itp(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    keyboard = [[InlineKeyboardButton("Python", callback_data='ITP python'),
                 InlineKeyboardButton("PHP + Python", callback_data='ITP php-python')]]
    imk = InlineKeyboardMarkup(keyboard)
    bot.send_message(chat_id=chat_id, text="""
Industrial Training Program is a 3 months program conducted on Python(with Adv Frameworks) and Python PHP
(Integration of two technologies).

Students will be given 1 month training and 2 month work on Live Project.

Information about ITP select option: """, reply_markup=imk)


# Give details about internship.
def internship(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    keyboard = [[InlineKeyboardButton("6 Month", callback_data='internship 6-month'),
                 InlineKeyboardButton("1 Year", callback_data='internship 1-year')]]
    imk = InlineKeyboardMarkup(keyboard)
    bot.send_message(chat_id=chat_id, text="""
Internship is given to Final Year Students who want do their final year project with the help of Foslipy.
There are two types of Internship -
6 months and 1 years.
Select Internship for: """, reply_markup=imk)


# Give details about Project Consultation.
def project_consultation(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    bot.send_message(chat_id=chat_id, text="""
Students who wants Support and Mentoring for completing their Final year project can approach.
we will give an industry touch to their project. We have Mentors who work on domains like Python Analytics, PHP
Advanced Framework Laravel, Cloud Computing, Android, Big Data and Hadoop and other open source technologies. 
Student will get to meet the mentor 2 times a month and for the next 3 months till Project is completed.
""")


# Give details about Interview Prepare.
def interview_prepare(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    bot.send_message(chat_id=chat_id, text="""
We will help students prepare for Interview. We will train them with
Aptitude - This will help students develop their Problem Solving Skills
Technical - This will help students build their Logical Thinking
Final HR - This will help students how to do negotiation and get selected.
""")


# Give details about Placement Assist.
def placement_assist(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    bot.send_message(chat_id=chat_id, text="""
We help students create and maintain the Developer Profile- GitHub Profile. 
We help students create and maintain the Professional Profile - Linkedin Profile.
We provide Work Experience certificate after completion of the Project.
Students will get 100% placement assistance.
""")


########################
# first message to bot #
########################

# Normal start to for students and staff.
def start(bot: Bot, update: Update):
    Thread(target=threaded_start, args=(bot, update)).start()


def threaded_start(bot: Bot, update: Update):
    chat_id = update.message.chat_id
    name = update.message.from_user.first_name
    try:
        assert update.message.from_user.username is not None, "Username is Not present"
        username = update.message.from_user.username.lower()
    except Exception as e:
        print(e)
        username = name + str(chat_id)
    print(username)
    print(time.asctime(time.localtime(time.time())))
    details = db.check_user(chat_id)
    if not details[0]:
        result = db.add_user(chat_id, name, username)
        if not result[0]:
            print("Unable to add user", result[1])
    bot.send_message(chat_id=chat_id, text="""This is Foslipy Bot.
/start - Subscribe to Bot.
/Workshop - to get details about Workshop.
/ITP - to get details about ITP.
/Internship - to get details about Internship.
/Project_Consultation - to get details about Project Consultation.
/Interview - to get details about Interview Prepare.
/Placement - to get details about Placement Assist.
/Chat_bot - to get details about Chat Bot.
""", reply_markup=reply_markup)


# Kick Start to bot.
def main():
    updater = Updater('881292301:AAFpIbGSdj6iG2UK1QQsZoajJ_xshIise1A')

    # if admin message the show admin's control panel menu.
    updater.dispatcher.add_handler(CommandHandler("cp", admin_start, filters=Filters.user(username='pyshivam')))

    # Normal Start
    updater.dispatcher.add_handler(CommandHandler("start", start))
    updater.dispatcher.add_handler(CommandHandler("Workshop", workshop))
    updater.dispatcher.add_handler(CommandHandler("ITP", itp))
    updater.dispatcher.add_handler(CommandHandler("Internship", internship))
    updater.dispatcher.add_handler(CommandHandler("Project_Consultation", project_consultation))
    updater.dispatcher.add_handler(CommandHandler("Interview", interview_prepare))
    updater.dispatcher.add_handler(CommandHandler("Placement", placement_assist))
    updater.dispatcher.add_handler(CommandHandler("Chat_bot", chat_bot))
    updater.dispatcher.add_handler(CallbackQueryHandler(button))
    updater.dispatcher.add_handler(MessageHandler(Filters.chat(chat_id=493497879), admin))
    updater.dispatcher.add_handler(MessageHandler(Filters.text, chat_bot))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    # Connection  to DataBase
    db = DataBase()

    # Kicking start to Program.
    main()

# 566519038:AAFab45Jz3GrGsV_t1qSwv8r4zTjikDHgKU
